In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 512kB 46.6MB/s 
     |████████████████████████████████| 3.8MB 32.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=cc56bce475083ea235dc5e1a14897b889874cc7c8b26a6153ba54230295d221d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [3]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [4]:
import numpy as np
from numpy import argmax

In [5]:
def get_char_embedd(char_encode):
  LEN_OF_VOCAB = 137
  shape = char_encode.shape
  char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      for k in range(shape[2]):
        char_int = char_encode[i,j,k]
        char_int = char_int.astype(np.int64)
        onehot = np.zeros(LEN_OF_VOCAB)
        onehot[char_int] = 1
        char_embedd[i,j,k,:] = onehot
  
  return char_embedd

In [6]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import math

In [7]:
# aphabet tag library
tags = {'pad': 0, 'LOCATION_HOMENUMBER': 1, 'LOCATION_STREET': 2, 'LOCATION_WARD': 3, 'LOCATION_DISTRICT': 4, 'LOCATION_PROVINCE': 5, 'LOCATION_COUNTRY': 6, 'LOCATION_POSTCODE': 7, 'LOCATION_NER': 8, 'OBJ': 9, 'OBJ_FEATURE': 10, 'PRE': 11, 'UNKNOW': 12 }
aphabet_tag = []
for key in tags:
  aphabet_tag.append(key)
# aphabet_tag = ['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW', 'LOCATION_SPECIAL']
print(aphabet_tag)

['pad', 'LOCATION_HOMENUMBER', 'LOCATION_STREET', 'LOCATION_WARD', 'LOCATION_DISTRICT', 'LOCATION_PROVINCE', 'LOCATION_COUNTRY', 'LOCATION_POSTCODE', 'LOCATION_NER', 'OBJ', 'OBJ_FEATURE', 'PRE', 'UNKNOW']


In [9]:
# load model=========================================================================
import os.path
from os import path

index = 0
link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)
while path.exists(link_model):
  print("model {}".format(index))
  index+=1
  link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)

index-=1
link_model = "/content/drive/MyDrive/model/model-data-{}.h5".format(index)
m = load_model(link_model)
#====================================================================================

model 0
model 1
model 2
model 3





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [10]:
import math

link_folder = "/content/drive/MyDrive/fast_text/split data/data_0"
index = 0
while path.exists(link_folder):
  print("DATA {}".format(index))
  
  link_val = "/content/drive/MyDrive/fast_text/split data/data_{}/val/".format(index)
  
  val_char_encode = np.loadtxt(link_val+'char_encode.txt')
  val_char_encode = val_char_encode.reshape(int(val_char_encode.shape[0]/42), 42, val_char_encode.shape[1])
  val_char_embedd = get_char_embedd(val_char_encode)
  print(val_char_embedd.shape)
  val_tag = np.loadtxt(link_val+'tag_embedd.txt')
  val_tag = val_tag.reshape(int(val_tag.shape[0]/42), 42, val_tag.shape[1])
  print(val_tag.shape)
  val_word = np.loadtxt(link_val+'word_embedd.txt')
  val_word = val_word.reshape(int(val_word.shape[0]/42), 42, val_word.shape[1])
  print(val_word.shape)

  y_pred = m.predict([val_char_embedd, val_word])
  y_true = val_tag
  print(y_pred.shape)
  print(y_true.shape)

  y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
  y_pred = np.argmax(y_pred,axis=1)
  print(y_pred.shape)

  y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
  y_true = np.argmax(y_true,axis=1)
  print(y_true.shape)

  # tìm acc
  a_true = 0
  a_false = 0
  for i in range(len(y_true)):
    if y_true[i]!=0 and y_true[i]!=12:
      if y_true[i]==y_pred[i]:
        a_true+=1
      else:
        a_false+=1
  
  acc = format(a_true/(a_true+a_false)*100, '.2f')
  print(' accuracy = {}'.format(acc))

  # lấy độ predict 
  dicts = []
  for element in y_true:
    if element not in dicts:
      dicts.append(element)
  dicts.sort()
  print(dicts)

  cnf_matrix = confusion_matrix(y_true, y_pred, labels=dicts)
  for i in range(len(cnf_matrix)):
    for j in range(len(cnf_matrix)):
      cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
  # print(cnf_matrix)

  # số tag đã đoán
  # print("số tag đã đoán của mỗi tag :")
  sum_doc = np.sum(cnf_matrix, axis = 0)
  num_pre = {}
  for i in range(len(cnf_matrix)):
    num_pre[aphabet_tag[i]] = sum_doc[i]
  # print(num_pre)

  # số tag đoán đúng
  # print("số tag đã đoán đúng của mỗi tag :")
  num_pre_true = {}
  for i in range(len(cnf_matrix)):
    num_pre_true[aphabet_tag[i]] = cnf_matrix[i][i]
  # print(num_pre_true)

  # số đối tượng có sẵn của mỗi tag
  # print("số tag có sẵn của mỗi tag : ")
  sum_ngang = np.sum(cnf_matrix, axis = 1)
  num_tag = {}
  for i in range(len(cnf_matrix)):
    num_tag[aphabet_tag[i]] = sum_ngang[i]
  # print(num_tag)
  
  # convert
  num_pre_cv = [ num_pre[tag] for tag in num_pre]
  num_pre_true_cv = [ num_pre_true[tag] for tag in num_pre_true]
  num_tag_cv = [ num_tag[tag] for tag in num_tag]

  X = np.empty([1, 3, 13])
  X[0, 0, :] = num_pre_true_cv
  X[0, 1, :] = num_pre_cv
  X[0, 2, :] = num_tag_cv
  fileout = open("/content/drive/MyDrive/model/evaluate/evaluate_val_data.txt", 'a+', encoding='utf8')
  np.savetxt(fileout, X[0])
  fileout.close()
  
  index+=1
  link_folder = "/content/drive/MyDrive/fast_text/split data/data_{}".format(index)

DATA 0
(450, 42, 32, 137)
(450, 42, 13)
(450, 42, 300)
(450, 42, 13)
(450, 42, 13)
(18900,)
(18900,)
 accuracy = 99.85
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
DATA 1
(450, 42, 32, 137)
(450, 42, 13)
(450, 42, 300)
(450, 42, 13)
(450, 42, 13)
(18900,)
(18900,)
 accuracy = 99.78
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
DATA 2
(450, 42, 32, 137)
(450, 42, 13)
(450, 42, 300)
(450, 42, 13)
(450, 42, 13)
(18900,)
(18900,)
 accuracy = 99.54
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
DATA 3
(375, 42, 32, 137)
(375, 42, 13)
(375, 42, 300)
(375, 42, 13)
(375, 42, 13)
(15750,)
(15750,)
 accuracy = 99.69
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
